In [1]:
#coding:utf-8
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_datata

In [2]:
#读入数据
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
# x为训练图像的占位符、y_为训练图像标签的占位符
x = tf.placeholder(tf.float32,[None,784])
y_ = tf.placeholder(tf.float32,[None,10])

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [4]:
#使用卷积网络，需要将784维还原为28x28的图片形式
x_image = tf.reshape(x,[-1,28,28,1])

x_image为输入图像，接下来对训练图像进行卷积运算。  
第一层卷积代码如下：

In [5]:
#创建卷积核
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

#创建偏置
def bias_variable(shape):
    initial = tf.constant(0.1,shape = shape)
    return tf.Variable(initial)

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#第一层卷积：卷积、激活函数、池化
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+ b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
#第二层卷积
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+ b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [7]:
#全连接层，输出为1024维向量
W_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
#使用Dropout , keep_prob是一个占位符，训练时为0.5，测试时为1
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [8]:
#再加一层全连接层，将h_fc1_drop由1024维的向量转换为10维，对应10个类别
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop,W_fc2)+b_fc2

In [10]:
#使用TensorFlow的方法直接计算交叉熵
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_,logits=y_conv))
#定义train_step
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [11]:
#定义测试的准确率
correct_prediction = tf.equal(tf.argmax(y_conv,1),tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [13]:
#创建session，对变量初始化
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

#训练20000步
for i in range(20000):
    batch = mnist.train.next_batch(50)
    #每100步报告一次在验证集上的准确率
    if i%100 ==0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0],y_:batch[1],keep_prob:1.0})
        print ("step %d,training accuracy %g" % (i,train_accuracy))
    train_step.run(feed_dict={x:batch[0],y_:batch[1],keep_prob:0.5})

F:\Anaconda\install\envs\tensorflow\lib\site-packages\tensorflow\python\client\session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


step 0,training accuracy 0.1
step 100,training accuracy 0.86
step 200,training accuracy 0.86
step 300,training accuracy 0.88
step 400,training accuracy 0.9
step 500,training accuracy 0.94
step 600,training accuracy 0.96
step 700,training accuracy 0.96
step 800,training accuracy 0.96
step 900,training accuracy 0.96
step 1000,training accuracy 0.94
step 1100,training accuracy 0.96
step 1200,training accuracy 1
step 1300,training accuracy 0.94


KeyboardInterrupt: 

In [ ]:
print("test accuracy %g" % accuracy.eval(feed_dict={x:mnist.test.images,y_:mnist.test.labels,keep_prob:1.0}))